In [36]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import config
import line_noti
import datetime
import pytz

tablist = config.tablist()
unit_dict = config.unit_dict()
npoints = config.npoints()

msg = "Water Report\n"

bangkok_tz = pytz.timezone('Asia/Bangkok')
now_utc = datetime.datetime.now(pytz.utc)
now_bangkok = now_utc.astimezone(bangkok_tz)
day_before = now_bangkok - datetime.timedelta(days=1)

run_date = now_bangkok.strftime(format="%Y-%m-%d")
run_date = '2024-06-13' #Uncomment here to override

date_end = run_date
date_start = ((datetime.datetime.strptime(date_end,"%Y-%m-%d")) + datetime.timedelta(days=-1)).strftime(format = "%Y-%m-%d")
msg += f"Run Date: {run_date}\n"

msg += "-"*24 +"\n"

In [37]:
date_start, date_end

('2024-06-12', '2024-06-13')

In [38]:
tabname = tablist[0]
npoints = config.npoints()

In [39]:
msg += f"Data from table {tabname}\n"

url = f'http://110.49.150.135:4002/CPU/?command=DataQuery&uri=dl:tabACTW&format=html&mode=date-range&p1={date_start}T18:00:00&p2={date_end}T18:00:00'
page = requests.get(url)
soup = BeautifulSoup(page.text, 'html.parser')

table = soup.find('table')
rows = table.find_all('tr')
data = []
for row in rows:
    cols = row.find_all(['td', 'th'])
    cols = [ele.text.strip() for ele in cols]
    data.append([ele for ele in cols if ele])

df = pd.DataFrame(data[1:],columns=data[0])

In [40]:
df.shape

(1440, 6)

In [41]:
collist= [col for col in df.columns if col not in ['TimeStamp','Record']]

df['TimeStamp'] = pd.to_datetime(df['TimeStamp']).dt.tz_localize(bangkok_tz)

In [42]:
df['TimeStamp'].min()

Timestamp('2024-06-12 18:00:00+0700', tz='Asia/Bangkok')

In [43]:
df['TimeStamp'].max()

Timestamp('2024-06-13 17:59:00+0700', tz='Asia/Bangkok')

In [44]:
msg += f"Number of data points: {df_period.shape[0]}\n\n"

for col in collist:
    df[col] = df[col].astype('float')
    msg += f"{col}: {df[col].median():.2f} {unit_dict.get(col) if unit_dict.get(col) is not None else ''}\n"

msg += "-"*24 + "\n"

In [45]:
print(msg)

Water Report
Run Date: 2024-06-13
------------------------
Data from table ACTW
Number of data points: 1153

WaterTemp_ACTW: 30.89 °C
Conductivity: 39.77 
Salinity: 22.29 psu
ACTWVoltage: 12.57 
------------------------

